# Developing an AI application to detect potholes 

This notebook works through 3 classifications of common objects on the road: stoplights, cars, and potholes. The Tesla 2020.12* software already includes cars, cones, stoplights/signs, humans, bikes, and road markings, but not potholes. 
YOLOV3 model info. A list of the models can be found here https://pytorch.org/docs/stable/torchvision/models.html_.

The project is broken down into multiple steps:
1. Extract frames from video footage
2. Define object classes within the images 



### Extract frames from video: 

- run extract_frame_video.py on each of the files
- TeslaCam records at 36 fps. I only utilize every 1 in 10 frames. 

### Define object classes and setup config files: 

- run the Python GUI in the director for each class
- draw boxes around the object and a file with coordinates is automatically written out with bbox.py
- create the train.txt and val.txt files with database_files.py. These contain the images for each set.
- get the YOLOv3 weights file with: wget https://pjreddie.com/media/files/yolov3.weights





In [8]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
#import torch.optim as optim
import torch.optim as opt






In [9]:
parser = argparse.ArgumentParser()
parser.add_argument("--epochs", type=int, default=20, help="number of epochs")
parser.add_argument("--image_folder", type=str, default="data/artifacts/images", help="path to dataset")
parser.add_argument("--batch_size", type=int, default=16, help="size of each image batch")
parser.add_argument("--model_config_path", type=str, default="config/yolov3.cfg", help="path to model config file")
parser.add_argument("--data_config_path", type=str, default="config/coco.data", help="path to data config file")
parser.add_argument("--weights_path", type=str, default="config/yolov3.weights", help="path to weights file")
parser.add_argument("--class_path", type=str, default="config/coco.names", help="path to class label file")
parser.add_argument("--conf_thres", type=float, default=0.8, help="object confidence threshold")
parser.add_argument("--nms_thres", type=float, default=0.4, help="iou thresshold for non-maximum suppression")
parser.add_argument("--n_cpu", type=int, default=0, help="number of cpu threads to use during batch generation")
parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
parser.add_argument("--checkpoint_interval", type=int, default=1, help="interval between saving model weights")
parser.add_argument("--checkpoint_dir", type=str, default="checkpoints", help="directory where model checkpoints are saved")
parser.add_argument("--use_cuda", type=bool, default=True, help="whether to use cuda if available")
opt = parser.parse_args()
print(opt)

usage: ipykernel_launcher.py [-h] [--epochs EPOCHS]
                             [--image_folder IMAGE_FOLDER]
                             [--batch_size BATCH_SIZE]
                             [--model_config_path MODEL_CONFIG_PATH]
                             [--data_config_path DATA_CONFIG_PATH]
                             [--weights_path WEIGHTS_PATH]
                             [--class_path CLASS_PATH]
                             [--conf_thres CONF_THRES] [--nms_thres NMS_THRES]
                             [--n_cpu N_CPU] [--img_size IMG_SIZE]
                             [--checkpoint_interval CHECKPOINT_INTERVAL]
                             [--checkpoint_dir CHECKPOINT_DIR]
                             [--use_cuda USE_CUDA]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/jennifercooper/Library/Jupyter/runtime/kernel-0672d384-4cb4-46d1-80bc-6d42a17cd7aa.json


SystemExit: 2

In [18]:
epochs     = 20
batch_size = 16
conf_thres = 0.8 #object confidence threshold
nms_thres  = 0.4 #iou thresshold for non-maximum suppression
img_size   = 416

checkpoint_interval = 1 #interval between saving model weights
checkpoint_dir      = 'data/checkpoint'

#paths to important files 
image_folder      = '/Users/jennifercooper/Projects/Tesla/PyTorchTraining/Potholes/data/artifacts/images'
model_config_path = '/Users/jennifercooper/Projects/Tesla/PyTorchTraining/Potholes/data/config/yolov3.cfg'
weights_path      = '/Users/jennifercooper/Projects/Tesla/PyTorchTraining/Potholes/data/config/yolov3.weights'
class_path        = '/Users/jennifercooper/Projects/Tesla/PyTorchTraining/Potholes/data/config/tesla_dashcam.names'
data_config_path  = '/Users/jennifercooper/Projects/Tesla/PyTorchTraining/Potholes/data/config/tesla_dashcam.data'

In [19]:
os.makedirs("checkpoints", exist_ok=True)

#classes = load_classes(opt.class_path)
classes = load_classes(class_path)

# Get data configuration
data_config = parse_data_config(opt.data_config_path)
train_path = data_config["train"]

# Get hyper parameters
hyperparams = parse_model_config(opt.model_config_path)[0]
learning_rate = float(hyperparams["learning_rate"])
momentum = float(hyperparams["momentum"])
decay = float(hyperparams["decay"])
burn_in = int(hyperparams["burn_in"])

# Initiate model
model = Darknet(opt.model_config_path)
model.load_weights(opt.weights_path)
#model.apply(weights_init_normal)

#if cuda:
#    model = model.cuda()
model.to('cpu')
model.train()


AttributeError: module 'torch.optim' has no attribute 'data_config_path'